## Data processing

In [0]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [0]:
data = pd.read_csv('prepared_data.csv')
data.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2283) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(56897, 2432)

In [0]:
data.describe()

,Unnamed: 0,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,pnr_user,trq_user,stm_user,sed_user,cig_user,alc_user,mj_user,coc_user,crack_user,her_user
count,56897.00000,56897.000000,56886.000000,56885.000000,56531.000000,56341.000000,56897.000000,56790.000000,56861.000000,56850.000000,...,56897.000000,56897.000000,56897.000000,56897.000000,56897.000000,56881.000000,56859.000000,56878.000000,56875.000000,56861.000000
mean,28448.00000,1.512751,78.090813,78.099077,519.117369,93.008857,47.982512,78.500229,92.979863,75.202375,...,0.049581,0.027734,0.030546,0.005695,0.437492,0.594961,0.179602,0.025247,0.003007,0.004397
std,16424.89347,0.499842,39.399555,39.385467,487.314111,12.930240,44.156738,28.116961,4.323097,34.456714,...,0.217079,0.164212,0.172087,0.075247,0.496082,0.490904,0.383859,0.238959,0.054750,0.066162
min,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14224.00000,1.000000,99.000000,99.000000,16.000000,91.000000,3.000000,91.000000,91.000000,91.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28448.00000,2.000000,99.000000,99.000000,991.000000,91.000000,91.000000,91.000000,91.000000,91.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,42672.00000,2.000000,99.000000,99.000000,991.000000,99.000000,91.000000,93.000000,93.000000,93.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,56896.00000,2.000000,99.000000,99.000000,991.000000,99.000000,91.000000,93.000000,99.000000,93.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,1.000000,1.000000


In [0]:
data.head()

,Unnamed: 0,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,pnr_user,trq_user,stm_user,sed_user,cig_user,alc_user,mj_user,coc_user,crack_user,her_user
0,0,1,99.0,99.0,16.0,99.0,4,93.0,93.0,93.0,...,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0
1,1,1,99.0,99.0,15.0,99.0,1,7.0,99.0,2.0,...,1,1,1,1,1,1.0,1.0,1.0,1.0,1.0
2,2,1,99.0,99.0,26.0,99.0,1,7.0,99.0,4.0,...,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0
3,3,2,4.0,4.0,991.0,91.0,91,91.0,91.0,91.0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,4,1,99.0,99.0,5.0,99.0,4,93.0,93.0,93.0,...,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0


#### Dealing with missing values
Due to the cpecifics of the survey data I wanted to use 2 types of missing value imputation. For all drug-related columns I used global most common substitution, because most peope do not use drugs. The rest columns with missing values were supposed to be imputed by the KNN model.

__BUT__

Unfortunately i can not afford to run it on so much data, so I used mode imputation in all dataset.

Columns with more than 20% of missing values are droped cince mode MVs imputation would influence data too much, and there are relatively not much of such columns and too much columns in a dataset anyway.

In [0]:
none_percs = data[data.columns[data.isnull().any()]].isnull().mean() * 100
nearly_empty_cols = [none_percs[none_percs == perc].index[0] for perc in none_percs if perc>20]
len(nearly_empty_cols)

357

In [0]:
data.drop(nearly_empty_cols, axis=1, inplace=True)

In [0]:
# number of cols with MVs, total part of MVs
len(data.columns[data.isnull().any()]),  data.isnull().values.ravel().sum()/(data.shape[0]*data.shape[1])

(1638, 0.059268863420781505)

##### Data Separation
I want to predict if a person uses particular drug using answers to the specific-drug-unrelated questions in the survey and then using questions not connected to any drug usage. For that I need to separate all drug related variables from general data.

##### Prefixes in variables-questions used for every type of drug:

In [0]:
halucinogens = ['LSD', 'PCP','PEYOTE','MESC','PSILCY','ECST','KET','DMTAMTFXY','SALVIADIV']
inhalants = ['AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER', 'SOLVENT', 'LGAS', 'NITOXID',
             'FELTMARKR', 'SPPAINT', 'AIRDUSTER', 'OTHAEROS']

In [0]:
prefixes = {}
prefixes['tobacco'] = 'CIG', 'CGR', 'CI3', 'PIP', 'SMK', 'TOB', 'CD'
prefixes['alcohol'] = ('AL')
prefixes['marijuana'] = 'MJ', 'MR'
prefixes['cocaine'] = 'COC', 'CC', 'CON'
prefixes['crack'] = 'CRK', 'CR' 
prefixes['heroin'] = 'HER', 'HR', 'HEO'
prefixes['hallucinogens'] = ['HAL']+halucinogens
prefixes['inhalants'] = ['INH']+inhalants
prefixes['methamphetamine'] = 'MET', 'ME'
prefixes['pain relievers'] = 'PNR', 'OXC'
prefixes['tranqualizers'] = ('TRQ')
prefixes['stimulants'] = 'SED', 'PSY'
prefixes['sedaives'] = ('STM')
prefixes['special drugs'] = 'COLD', 'GH', 'OT'
prefixes['general about drugs'] = ('GNN')
prefixes['targets'] = 'hal_user', 'inh_user', 'meth_user', 'pnr_user', 'trq_user', 'stm_user', 'sed_user', 'cig_user','alc_user', 'mj_user', 'coc_user', 'crack_user', 'her_user'

In [0]:
drug_cols = []
for drug in prefixes.keys():
  drug_cols += [col for col in data.columns if col.startswith(tuple(prefixes[drug]))]
len(drug_cols)

1800

In [0]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
data[data.columns] = imputer.fit_transform(data[data.columns])

In [0]:
data.isnull().values.ravel().sum()

0

In [0]:
data.to_csv('mode_imputed.csv')

#### Dealing with noise

#### Data reduction
i used Factor Analysis as it not only reduces dimentions but may increase accuracy

In [0]:
    fa = FactorAnalysis(n_components=300)
    fa.fit(train_x.values)
    train_set = fa.transform(train_x.values)
    test_set = fa.transform(test_x.values)

influence of RSK

special drugs

needle any